# Python 实验3（进阶）

## 使用海龟绘制给定照片或艺术作品

### 用到的外部库

In [1]:
import turtle as tt
from bs4 import BeautifulSoup
import numpy as np
import cv2
import os
from typing import Generator, Any
from tqdm import tqdm

### 一些常量与全局变量

In [2]:
# 贝塞尔函数（英文：Bézier function，后文略）的取样次数。
WRITE_STEP = 15

# 界面宽度。
WIDTH = 600

# 界面高度。
HEIGHT = 600

# 记录前一个贝塞尔函数的句柄。
XH = 0
YH = 0

# 用于转换 SVG 数据时记录放缩因子。
SCALE = (1, 1)

# 是否首次调用 SVG 数据处理过程。
FIRST = True

# 记录绘制过程中要处理多少种颜色。
K = 32

# 是否禁用绘制动画。
DISABLE_ANIMATION = False

### 贝塞尔函数

贝塞尔函数：用于生成贝塞尔曲线。[什么是贝塞尔曲线](https://zhuanlan.zhihu.com/p/688186803)



In [3]:
def bezier(p1: float, p2: float, t: float) -> float:
    """
    一阶贝塞尔函数。
    :param p1:
    :param p2:
    :param t:
    :return:
    """

    return p1 * (1 - t) + p2 * t


def bezier_2(
        x1: float, y1: float,
        x2: float, y2: float,
        x3: float, y3: float
) -> None:
    """
    二阶贝塞尔函数。
    :param x1:
    :param y1:
    :param x2:
    :param y2:
    :param x3:
    :param y3:
    :return:
    """

    tt.goto(x1, y1)
    tt.pd()
    for t in range(0, WRITE_STEP + 1):
        x = bezier(bezier(x1, x2, t / WRITE_STEP),
                   bezier(x2, x3, t / WRITE_STEP),
                   t / WRITE_STEP)
        y = bezier(bezier(y1, y2, t / WRITE_STEP),
                   bezier(y2, y3, t / WRITE_STEP),
                   t / WRITE_STEP)
        tt.goto(x, y)
    tt.pu()


def bezier_3(
        x1: float, y1: float,
        x2: float, y2: float,
        x3: float, y3: float,
        x4: float, y4: float
) -> None:
    """
    三阶贝塞尔函数。
    :param x1:
    :param y1:
    :param x2:
    :param y2:
    :param x3:
    :param y3:
    :param x4:
    :param y4:
    :return:
    """

    # 坐标变换
    x1 = -WIDTH / 2 + x1
    y1 = HEIGHT / 2 - y1
    x2 = -WIDTH / 2 + x2
    y2 = HEIGHT / 2 - y2
    x3 = -WIDTH / 2 + x3
    y3 = HEIGHT / 2 - y3
    x4 = -WIDTH / 2 + x4
    y4 = HEIGHT / 2 - y4

    tt.goto(x1, y1)
    tt.pd()
    for t in range(0, WRITE_STEP + 1):
        x = bezier(bezier(bezier(x1, x2, t / WRITE_STEP),
                          bezier(x2, x3, t / WRITE_STEP),
                          t / WRITE_STEP),
                   bezier(bezier(x2, x3, t / WRITE_STEP),
                          bezier(x3, x4, t / WRITE_STEP),
                          t / WRITE_STEP),
                   t / WRITE_STEP)
        y = bezier(bezier(bezier(y1, y2, t / WRITE_STEP),
                          bezier(y2, y3, t / WRITE_STEP),
                          t / WRITE_STEP),
                   bezier(bezier(y2, y3, t / WRITE_STEP),
                          bezier(y3, y4, t / WRITE_STEP),
                          t / WRITE_STEP),
                   t / WRITE_STEP)
        tt.goto(x, y)
    tt.pu()

### 基于上述贝塞尔函数来绘制矢量图曲线

In [4]:
def move_to(x: float, y: float) -> None:
    """
    移动到 SVG 坐标下（x，y）。
    :param x:
    :param y:
    :return:
    """

    tt.pu()
    tt.goto(-WIDTH / 2 + x, HEIGHT / 2 - y)
    tt.pd()


def move_to_r(dx: float, dy: float) -> None:
    """
    移动到 SVG 当前相对坐标下（dx，dy）。
    :param dx:
    :param dy:
    :return:
    """

    tt.pu()
    tt.goto(tt.xcor() + dx, tt.ycor() - dy)
    tt.pd()


def line(x1: float, y1: float, x2: float, y2: float) -> None:
    """
    连接 SVG 坐标下两点。
    :param x1:
    :param y1:
    :param x2:
    :param y2:
    :return:
    """

    tt.pu()
    tt.goto(-WIDTH / 2 + x1, HEIGHT / 2 - y1)
    tt.pd()
    tt.goto(-WIDTH / 2 + x2, HEIGHT / 2 - y2)
    tt.pu()


def line_to_r(dx: float, dy: float) -> None:
    """
    连接当前点和相对坐标（dx，dy）的点。
    :param dx:
    :param dy:
    :return:
    """

    tt.pd()
    tt.goto(tt.xcor() + dx, tt.ycor() - dy)
    tt.pu()


def line_to(x: float, y: float) -> None:
    """
    连接当前点和 SVG 坐标下（x，y）的点。
    :param x:
    :param y:
    :return:
    """

    tt.pd()
    tt.goto(-WIDTH / 2 + x, HEIGHT / 2 - y)
    tt.pu()


def curve_to(
        x1: float, y1: float,
        x2: float, y2: float,
        x: float, y: float
) -> None:
    """
    三阶贝塞尔曲线到（x，y）。
    :param x1:
    :param y1:
    :param x2:
    :param y2:
    :param x:
    :param y:
    :return:
    """

    global XH, YH

    tt.pu()
    x_now = tt.xcor() + WIDTH / 2
    y_now = HEIGHT / 2 - tt.ycor()
    bezier_3(x_now, y_now, x1, y1, x2, y2, x, y)
    XH = x - x2
    YH = y - y2


def curve_to_r(
        x1: float, y1: float,
        x2: float, y2: float,
        x: float, y: float
) -> None:
    """
    # 三阶贝塞尔曲线到相对坐标（x，y）。
    :param x1:
    :param y1:
    :param x2:
    :param y2:
    :param x:
    :param y:
    :return:
    """

    global XH, YH

    tt.pu()
    x_now = tt.xcor() + WIDTH / 2
    y_now = HEIGHT / 2 - tt.ycor()
    bezier_3(x_now, y_now, x_now + x1, y_now + y1, x_now + x2, y_now + y2, x_now + x, y_now + y)
    XH = x - x2
    YH = y - y2

### 用于处理 SVG 图片数据的辅助函数

In [5]:
def transform(w_attr: str) -> None:
    global SCALE

    funcs = w_attr.split(' ')
    for func in funcs:
        func_name = func[0:func.find('(')]
        if func_name == 'scale':
            SCALE = (float(func[func.find('(') + 1:-1].split(',')[0]),
                     -float(func[func.find('(') + 1:-1].split(',')[1]))


def read_path_attr_d(w_attr: str) -> Generator[float, Any, None]:
    ulist = w_attr.split(' ')
    for i in ulist:
        if len(i) == 0:
            continue
        elif i.isdigit() or i.isalpha():
            yield float(i)
        elif i[0].isalpha():
            yield i[0]
            yield float(i[1:])
        elif i[-1].isalpha():
            yield float(i[0:-1])
        elif i[0] == '-':
            yield float(i)

### 绘制 SVG 矢量图

定义绘制单一颜色 SVG 矢量图的函数。

In [6]:
def draw_svg(filename: str, w_color: str) -> None:
    global FIRST

    with open(filename, 'r') as svg_file:
        svg = BeautifulSoup(svg_file.read(), 'lxml')
        height = float(svg.svg.attrs['height'][0:-2])
        width = float(svg.svg.attrs['width'][0:-2])
        transform(svg.g.attrs['transform'])
        if FIRST:
            tt.setup(height=height, width=width)
            tt.setworldcoordinates(-width / 2 + 300, 300, width / 2 + 300, -height + 300)
            tt.mode("world")
            FIRST = False
        if DISABLE_ANIMATION:
            tt.tracer(False)
            tt.speed(0)
            tt.delay(0)
        else:
            tt.speed("fast")
            tt.tracer(n=1000, delay=10)
        tt.pensize(1)
        tt.pu()
        tt.color(w_color)
        tt.shape("turtle")

        paths = svg.find_all('path')
        for idx, i in enumerate(paths):
            print(f"正在绘制当前色层的第{idx}个path，总共{len(paths)}个")
            attr = i.attrs['d'].replace('\n', ' ')
            f = read_path_attr_d(attr)
            last_j = ''
            for j in tqdm(f, desc='正在绘制当前path'):
                match j:
                    case 'M':
                        tt.end_fill()
                        move_to(next(f) * SCALE[0], next(f) * SCALE[1])
                        tt.begin_fill()
                    case 'm':
                        tt.end_fill()
                        move_to_r(next(f) * SCALE[0], next(f) * SCALE[1])
                        tt.begin_fill()
                    case 'C':
                        curve_to(next(f) * SCALE[0], next(f) * SCALE[1],
                                 next(f) * SCALE[0], next(f) * SCALE[1],
                                 next(f) * SCALE[0], next(f) * SCALE[1])
                        last_j = j
                    case 'c':
                        curve_to_r(next(f) * SCALE[0], next(f) * SCALE[1],
                                   next(f) * SCALE[0], next(f) * SCALE[1],
                                   next(f) * SCALE[0], next(f) * SCALE[1])
                        last_j = j
                    case 'L':
                        line_to(next(f) * SCALE[0], next(f) * SCALE[1])
                    case 'l':
                        line_to_r(next(f) * SCALE[0], next(f) * SCALE[1])
                        last_j = j
                    case _:
                        match last_j:
                            case 'C':
                                curve_to(j * SCALE[0], next(f) * SCALE[1],
                                         next(f) * SCALE[0], next(f) * SCALE[1],
                                         next(f) * SCALE[0], next(f) * SCALE[1])
                            case 'c':
                                curve_to_r(j * SCALE[0], next(f) * SCALE[1],
                                           next(f) * SCALE[0], next(f) * SCALE[1],
                                           next(f) * SCALE[0], next(f) * SCALE[1])
                            case 'L':
                                line_to(j * SCALE[0], next(f) * SCALE[1])
                            case 'l':
                                line_to_r(j * SCALE[0], next(f) * SCALE[1])
            tt.pu()
            tt.update()

### 将给定的一整张位图数据使用 OpenCV 进行处理然后绘制

将给定的一整张位图数据使用 OpenCV 进行处理，分成不同部分的近似单一颜色区域，交由 potrace 处理为矢量图，然后根据所得矢量数据调用海龟进行绘制。

In [7]:
def draw_bitmap(w_image: cv2.typing.MatLike) -> None:
    print('Reducing the colors...')
    z = w_image.reshape((-1, 3))

    # convert to np.float32
    z = np.float32(z)

    # define criteria, number of clusters(K) and apply kmeans()
    criteria = (cv2.TERM_CRITERIA_EPS, 10, 1.0)
    print('begin kmeans')
    ret, label, center = cv2.kmeans(z, K, None, criteria, 10, cv2.KMEANS_RANDOM_CENTERS)
    print('end kmeans')

    # Now convert back into uint8, and make original image
    center = np.uint8(center)
    res = center[label.flatten()]
    res = res.reshape(w_image.shape)
    no = 0
    if os.path.exists('./.tmp.bmp'):
        os.remove('./.tmp.bmp')
    if os.path.exists('./.tmp.svg'):
        os.remove('./.tmp.svg')
    for i in center:
        print(f"正在绘制第{no}种颜色区域 ({no}/{K}) ...")
        no += 1
        res2 = cv2.inRange(res, i, i)
        res2 = cv2.bitwise_not(res2)
        cv2.imwrite('.tmp.bmp', res2)
        os.system('potrace .tmp.bmp -s --flat -o .tmp.svg')
        try:
            draw_svg('.tmp.svg', '#%02x%02x%02x' % (i[2], i[1], i[0]))
        finally:
            os.remove('.tmp.bmp')
            os.remove('.tmp.svg')
    tt.hideturtle()
    print('Done.')

### 测试我们的代码

In [14]:
print(os.getcwd())
bitmap = cv2.imread('4.jpg')

/home/srcres/Coding/Projects/python-learn/experiments/2


In [15]:
cv2.imshow('preview', bitmap)
cv2.waitKey(0)
cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is not the object's thread (0x1c866d00).
Cannot move to target thread (0x1c7ffb00)

QObject::moveToThread: Current thread (0x1c7ffb00) is n

In [18]:
# 先重置一下常量与全局变量。

# 贝塞尔函数（英文：Bézier function，后文略）的取样次数。
WRITE_STEP = 16

# 界面宽度。
WIDTH = 600

# 界面高度。
HEIGHT = 600

# 记录前一个贝塞尔函数的句柄。
XH = 0
YH = 0

# 用于转换 SVG 数据时记录放缩因子。
SCALE = (1, 1)

# 是否首次调用 SVG 数据处理过程。
FIRST = True

# 记录绘制过程中要处理多少种颜色。
K = 20

# 是否禁用绘制动画。
DISABLE_ANIMATION = False

In [19]:
# 先清屏
tt.clearscreen()
# 再调用海龟进行绘图
draw_bitmap(bitmap)

Reducing the colors...
begin kmeans


/tmp/ipykernel_37193/4274890544.py:5: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  svg = BeautifulSoup(svg_file.read(), 'lxml')


end kmeans
正在绘制第0种颜色区域 (0/20) ...
正在绘制当前色层的第0个path，总共1个


正在绘制当前path: 9758it [00:03, 2850.03it/s]


正在绘制第1种颜色区域 (1/20) ...
正在绘制当前色层的第0个path，总共1个


...

正在绘制第18种颜色区域 (18/20) ...
正在绘制当前色层的第0个path，总共1个


正在绘制当前path: 636it [00:04, 150.83it/s]


正在绘制第19种颜色区域 (19/20) ...
正在绘制当前色层的第0个path，总共1个


正在绘制当前path: 2628it [00:18, 140.14it/s]

Done.
